In [ ]:
import pulp as pl
import pandas as pd
import xlrd as xl
import numpy as np
while True:
  Instancia= input("Ingrese un numero del 1 al 6 para seleccionar la instancia a leer:")
  if Instancia == "1" or Instancia == "2" or Instancia == "3" or Instancia == "4" or Instancia == "5" or Instancia == "6":
    Periodos_congelados = input("Seleccione 1 si quiere observar una implementacion de horizonte rodante congelando cada 1 periodo, 2 para cada 2 periodos:")
    if Periodos_congelados == "1" or Periodos_congelados == "2":
      break
    else:
        print("Implementacion incorrecta, pruebe con 1 o 2")
  else:
    print("Instancia ingresada incorrecta, pruebe con un numero del 1 al 6")
print ("Instancia")
print (Instancia)
print ("Periodos_congelados")
print (Periodos_congelados)
if Instancia == "1":
  excel = "Instancia 1.xlsx"
elif Instancia == "2":
  excel = "Instancia 2.xlsx"
elif Instancia == "3":
  excel = "Instancia 3.xlsx"
elif Instancia == "4":
  excel = "Instancia 4.xlsx"
elif Instancia == "5":
  excel = "Instancia 5.xlsx"
elif Instancia == "6":
  excel = "Instancia 6.xlsx"
if Periodos_congelados == "1":
  Demanda_F= "Demanda F=1"
else:
  Demanda_F= "Demanda F=2"
wb = xl.open_workbook(excel)
sheet = wb.sheet_by_name(Demanda_F)
parametros = wb.sheet_by_name("Parametros")
periodo_actual=0
salida=0
while salida==0:
  print("periodo actual")
  print(periodo_actual)
  def reg_lineal(periodos,demandas1,demandas2,demandas3,periodo_actual):
    X = 12+periodo_actual
    Y1 = 0
    Y2 = 0
    Y3 = 0
    xy1=0
    xy2=0
    xy3=0
    suma_cuadx = 0
    for k in range (0,12+periodo_actual):
      xy1+=periodos[k]*demandas1[k]
      xy2+=periodos[k]*demandas2[k]
      xy3+=periodos[k]*demandas3[k]
      suma_cuadx += periodos[k]**2
      X+= k
      Y1 += demandas1[k]
      Y2 += demandas1[k]
      Y3 += demandas1[k]
    b1 = (periodos[11+periodo_actual]*xy1-(X*Y1))/(periodos[11+periodo_actual]*suma_cuadx-X**2)
    b2 = (periodos[11+periodo_actual]*xy2-(X*Y2))/(periodos[11+periodo_actual]*suma_cuadx-X**2)
    b3 = (periodos[11+periodo_actual]*xy3-(X*Y3))/(periodos[11+periodo_actual]*suma_cuadx-X**2)
    a1 = Y1/periodos[11+periodo_actual] - b1*X/periodos[11+periodo_actual]
    a2 = Y2/periodos[11+periodo_actual] - b2*X/periodos[11+periodo_actual]
    a3 = Y3/periodos[11+periodo_actual] - b3*X/periodos[11+periodo_actual]
    return a1,a2,a3,b1,b2,b3
  "-----------------------------------------------------------------------------------------------------------------------"
  def modelo_matematico(I,L,T,F_it,Ch_i,Cso_i,Cap_l,Cs_il,Cp_il,Ts_il,Tp_il,P_it,B_lt,M,Inv0_i,SO0_it,Mp0_ilt,S0_it,Z0_ilt):
    "Definir nombre del problema"

    prob = pl.LpProblem("APP", pl.LpMaximize)

    "Variables"

    Inv_it = pl.LpVariable.dicts("Inv_it[i,t]", [(i,t) for i in I for t in T], 0, None, pl.LpContinuous)#Incluye restriccion (7)

    Mp_ilt= pl.LpVariable.dicts("Mp_ilt[i,l,t]", [(i,l,t) for i in I for l in L for t in T], 0, None, pl.LpContinuous)

    S_it= pl.LpVariable.dicts("S_it[i,t]", [(i,t) for i in I for t in T], 0, None, pl.LpContinuous)

    Z_ilt= pl.LpVariable.dicts("Z_ilt[i,l,t]", [(i,l,t) for i in I for l in L for t in T], 0, 1, pl.LpBinary)#Incluye restriccion (8)

    SO_it= pl.LpVariable.dicts("SO_it[i,t]", [(i,t) for i in I for t in T], 0, None, pl.LpContinuous)


    prob += ((pl.lpSum(P_it[i,t]*S_it[i,t] - Ch_i[i]*Inv_it[i,t] - Cso_i[i]*SO_it[i,t] for i in I for t in T)) - (pl.lpSum(Cs_il[i,l]*Z_ilt[i,l,t] + Cp_il[i,l]*Mp_ilt[i,l,t]  for i in I for l in L for t in T))),"Z"


    #Restricciones para que las variables tengan los valores que el optimizador tomo en el pasado
    for i in I:
        for l in L:
            for t in T:
                if t <= periodo_actual+12:
                    prob += Z_ilt[i,l,t] == Z0_ilt[i,l,t]
                    prob += Mp_ilt[i,l,t] == Mp0_ilt[i,l,t]
    for i in I:
        for t in T:
            if t <= periodo_actual+12:
                prob += Inv_it[i,t] == Inv0_i[i,t]
                prob += S_it[i,t] == S0_it[i,t]
                prob += SO_it[i,t] == SO0_it[i,t]

    "Restricciones"

    for l in L:
        for t in T:
            prob += (pl.lpSum(Ts_il[i,l]*Z_ilt[i,l,t] + Tp_il[i,l]*Mp_ilt[i,l,t] for i in I)) <=  Cap_l[l]*B_lt[l,t]


    for i in I:
        for l in L:
            for t in T:
                prob += Mp_ilt[i,l,t] <= M*Z_ilt[i,l,t]

    for i in I:
        for l in L:
            for t in T:
                prob += Z_ilt[i,l,t] <= Mp_ilt[i,l,t]
    for i in I:
        for t in T:
            if t>13:
                prob += Inv_it[i,t] == Inv_it[i,t-1] + (pl.lpSum(Mp_ilt[i,l,t] for l in L)) -S_it[i,t]
            else:
                prob += Inv_it[i,t] == Inv0_i[i,t-1] + (pl.lpSum(Mp_ilt[i,l,t] for l in L)) -S_it[i,t] #Uso de inventario inicial
    for i in I:
        for t in T:
            prob += S_it[i,t] == F_it[i,t] - SO_it[i,t]
    status = prob.solve()
    return prob,Inv_it,status,SO_it,Mp_ilt,S_it,Z_ilt
  "------------------------------------------------------------------------------------------------------------------"

  "Conjuntos"

  I=[] #Productos
  T=[]   #Periodos
  L=[] #Lineas de produccion

  for i in range(0,3):
    I.append(parametros.cell_value(i+1,3))
    L.append(parametros.cell_value(i+3,1))
  for i in range(0,12):
    T.append(parametros.cell_value(20,4+i))



  "Parametros"

  F_it={}


  Cs_il = {}

  Ch_i = {}

  Cp_il = {}

  Cso_i = {}

  Ts_il = {}

  Cap_l = {}

  Tp_il = {}
  x = [11+periodo_actual,12+periodo_actual]
  if periodo_actual == 0:
    Inv0_i = {}
    SO0_it = {}
    Mp0_ilt = {}
    S0_it = {}
    Z0_ilt = {}
  else:
    if Periodos_congelados == "2": #Para los dos casos es distinto,
      for i in range(0,3):
        for j in range(0,2):
          Inv0_i[I[i],x[j]]=Inv_it[I[i],x[j]].value() #Variables de los periodos que ya pasaron
          SO0_it[I[i],x[j]]=SO_it[I[i],x[j]].value()
          for l in range (0,3):
            Z0_ilt[I[i],L[l],x[j]]=Z_ilt[I[i],L[l],x[j]].value()
            Mp0_ilt[I[i],L[l],x[j]]=Mp_ilt[I[i],L[l],x[j]].value()
            if Z0_ilt[I[i],L[l],x[j]] >0:
              Z0_ilt[I[i],L[l],x[j]] = 1
    elif Periodos_congelados == "1":
      for i in range(0,3):
        Inv0_i[I[i],x[1]]=Inv_it[I[i],x[1]].value()
        SO0_it[I[i],x[1]]=SO_it[I[i],x[1]].value()
        for l in range (0,3):
          Z0_ilt[I[i],L[l],x[1]]=Z_ilt[I[i],L[l],x[1]].value()
          Mp0_ilt[I[i],L[l],x[1]]=Mp_ilt[I[i],L[l],x[1]].value()
  P_it = {}

  B_lt = {}
  M = int(parametros.cell_value(1,11))
  for k in range (0,3):
    Ch_i[I[k]]=(float(parametros.cell_value(k+1,9)))
    Cso_i[I[k]]=(float(parametros.cell_value(k+6,9)))
    Cap_l[L[k]]=(float(parametros.cell_value(k+11,9)))
    if periodo_actual == 0:
      Inv0_i[I[k],x[1]]=(float(parametros.cell_value(k+16,9)))
    for v in range (0,12):
      if v <= 2:
        Cs_il[I[k],L[v]] = float(parametros.cell_value(k+1,v+4))
        Cp_il[I[k],L[v]] = float(parametros.cell_value(6+k,v+4))
        Ts_il[I[k],L[v]] = float(parametros.cell_value(11+k,v+4))
        Tp_il[I[k],L[v]] = float(parametros.cell_value(16+k,v+4))
      P_it[I[k],T[v]] = float(parametros.cell_value(21+k,v+4))
      B_lt[L[k],T[v]] = float(parametros.cell_value(26+k,v+4))
  demandas1 = []
  demandas2 = []
  demandas3 = []
  periodos = []
  demanda_dict = {}
  for j in range (0,24):
      periodos.append(int(sheet.cell_value(0,j+1)))
      demandas1.append(int(sheet.cell_value(1,j+1)))
      demandas2.append(int(sheet.cell_value(2,j+1)))
      demandas3.append(int(sheet.cell_value(3,j+1)))
  a1,a2,a3,b1,b2,b3 = reg_lineal(periodos,demandas1,demandas2,demandas3,periodo_actual)
  a = [a1,a2,a3]
  b = [b1,b2,b3]
  for i in range(0,3):
    for t in range (0,12):
      F_it[I[i],T[t]] = a[i] + b[i]*int(T[t])#Definicion de demanda estimada
      if periodo_actual + 12 >= t+13:
        demanda_dict[I[i],T[t]] = int(sheet.cell_value(i+1,t+13))#Demanda real de periodos ya transcurridos

  if periodo_actual>0:
    for i in range(0,3):
      for t in range(0,periodo_actual):
        if demanda_dict[I[i],T[t]]- SO0_it[I[i],T[t]]< 0:#En caso de que el stockout sea mayor a la demanda real, se vende 0 producto.
          S0_it[I[i],T[t]] = 0
        if F_it[I[i],T[t]] > demanda_dict[I[i],T[t]]: #cantidad de producto realmente vendida, siempre debe ser igual o menor a la demanda real
          S0_it[I[i],T[t]] = demanda_dict[I[i],T[t]]- SO0_it[I[i],T[t]]
        else:
          S0_it[I[i],T[t]] = F_it[I[i],T[t]]- SO0_it[I[i],T[t]]
        if t == 0:
          Inv0_i[I[i],T[t]] = Inv0_i[I[i],12] - S0_it[I[i],T[t]]+ Mp0_ilt[I[i],L[0],T[t]]  + Mp0_ilt[I[i],L[1],T[t]]  + Mp0_ilt[I[i],L[2],T[t]]#actualizacion de inventario a partir de ventas reales
        elif t == x[1]:
          Inv0_i[I[i],T[t]] = Inv0_i[I[i],T[t-1]] - S0_it[I[i],T[t]]+ Mp0_ilt[I[i],L[0],T[t]]  + Mp0_ilt[I[i],L[1],T[t]]  + Mp0_ilt[I[i],L[2],T[t]]#actualizacion de inventario a partir de ventas reales
        if Periodos_congelados =="2" and t == x[0]:
          Inv0_i[I[i],T[t]] = Inv0_i[I[i],T[t-1]] - S0_it[I[i],T[t]]+ Mp0_ilt[I[i],L[0],T[t]]  + Mp0_ilt[I[i],L[1],T[t]]  + Mp0_ilt[I[i],L[2],T[t]]
    F_it.update(demanda_dict)#Si ya transcurrieron periodos se cambia la demanda estimada de estos por la demanda real
  "Resolver el modelo"

  prob,Inv_it,status,SO_it,Mp_ilt,S_it,Z_ilt = modelo_matematico(I,L,T,F_it,Ch_i,Cso_i,Cap_l,Cs_il,Cp_il,Ts_il,Tp_il,P_it,B_lt,M,Inv0_i,SO0_it,Mp0_ilt,S0_it,Z0_ilt)
  op = pl.value(prob.objective)
  s = pl.LpStatus[status]

  print(s)
  for v in prob.variables():
    print(v.name,"=",v.varValue)

  print(op)
  print("---------------------------------------------------------------------------------------")
  print("")


  if periodo_actual == 11:
    salida=11
    break
  elif Periodos_congelados=="2" and periodo_actual == 10:
      salida=1
      break
  if Periodos_congelados == "1":
      periodo_actual+=1
  else:
      periodo_actual +=2


ModuleNotFoundError: ignored

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
